In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()
#findspark.add_packages('mysql:mysql-connector-java:8.0.11')

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:15 http

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-01-25 16:33:30--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-01-25 16:33:31 (6.55 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

**Load Amazon Data into Spark Dataframe**

In [4]:
from pyspark import SparkFiles
url = "https://mckenziek-bucket.s3.us-east-2.amazonaws.com/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
review_df =  spark.read.csv(SparkFiles.get("amazon_reviews_us_Apparel_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [7]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
amazon_df = review_df
amazon_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [8]:
# Create the customers_table DataFrame
customers_df = amazon_df.groupby("customer_id").agg({'customer_id':'count'}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   13780696|             1|
|   48489393|             8|
|   43380682|             1|
|   17133428|             1|
|    9731896|             2|
+-----------+--------------+
only showing top 5 rows



In [31]:
# Create the products_table DataFrame and drop duplicates. 
products_df = amazon_df.select(["product_id", "product_title"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B01KL6O72Y|Easy Tool Stainle...|
|B01ID3ZS5W|V28 Women Cowl Ne...|
|B01I497BGY|James Fiallo Men'...|
|B01HDXFZK6|Belfry Gangster 1...|
|B01G6MBEBY|JAEDEN Women's Be...|
|B01FWRXN0Y|Levi's Boys' 514 ...|
|B01EXNH1HE|Minimalist Wallet...|
|B01E7OL09O|Harriton Men's Ba...|
|B01DXHX81O|Jockey Women's Un...|
|B01DDULIJK|Alexander Del Ros...|
|B01BOKOL4A|Jockey Women's Un...|
|B01B3Q4Q0O|Bali Passion For ...|
|B01ADDSL9U|Lilyette Women's ...|
|B019P1X5XI|Vanity Fair Women...|
|B019MDXIXG|Warner's Women's ...|
|B019438FEG|Columbia Women's ...|
|B0178HGNIA|Robes King RK Cla...|
|B016VIU0QI|VIV Collection Be...|
|B016PUU3VO|Fruit of the Loom...|
|B016AQNDM4|Jockey Scrubs Wom...|
+----------+--------------------+
only showing top 20 rows



In [32]:
products_df.dtypes

[('product_id', 'string'), ('product_title', 'string')]

In [ ]:
#products_df.count()

In [37]:
products_df = amazon_df.select(["product_id", "product_title"]).drop_duplicates()
products_df.count()

2305630

In [20]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = amazon_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1KKOXHNI8MSXU|   32158956|B01KL6O72Y|      24485154| 2013-01-14|
|R26SP2OPDK4HT7|    2714559|B01ID3ZS5W|     363128556| 2014-03-04|
| RWQEDYAX373I1|   12608825|B01I497BGY|     811958549| 2015-07-12|
|R231YI7R4GPF6J|   25482800|B01HDXFZK6|     692205728| 2015-06-03|
|R3KO3W45DD0L1K|    9310286|B01G6MBEBY|     431150422| 2015-06-12|
|R1C4QH63NFL5NJ|   26631939|B01FWRXN0Y|     366144407| 2014-04-22|
|R2GP65O1U9N7BP|   48785098|B01EXNH1HE|     786052021| 2015-07-28|
|R3O29CT5MQQ3XQ|   39548589|B01E7OL09O|     108920964| 2015-07-10|
|R1ZECD2AA8QFF6|   29355866|B01DXHX81O|     317132458| 2015-08-09|
|R2S79GCF6J89OA|   27477484|B01DDULIJK|     110598191| 2014-05-24|
|R1CBCUGNP37MPK|   17685865|B01BOKOL4A|     121408017| 2015-06-10|
|R3NU4WUR5ZNV1V|   19787539|B01B3Q4Q0O|     134506082| 2013-09

In [21]:
# Create the vine_table. DataFrame
vine_df = amazon_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1KKOXHNI8MSXU|          4|            0|          0|   N|                Y|
|R26SP2OPDK4HT7|          5|            1|          2|   N|                Y|
| RWQEDYAX373I1|          5|            0|          0|   N|                Y|
|R231YI7R4GPF6J|          5|            0|          0|   N|                Y|
|R3KO3W45DD0L1K|          5|            0|          0|   N|                Y|
|R1C4QH63NFL5NJ|          5|            0|          0|   N|                Y|
|R2GP65O1U9N7BP|          5|            0|          0|   N|                Y|
|R3O29CT5MQQ3XQ|          4|            0|          0|   N|                Y|
|R1ZECD2AA8QFF6|          5|            0|          0|   N|                Y|
|R2S79GCF6J89OA|          3|            0|          0|   N|     

In [22]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://datavine.coqohurkyryc.us-east-2.rds.amazonaws.com:5432/vinedb"
config = {"user":"postgres",
          "password": "Nickaleuslane34&",
          "driver":"org.postgresql.Driver"}

In [23]:
# Write review_id_df to table in RDS
# review_id_df.write.jdbc(url=jdbc_url, table='review_id_table',mode=mode, properties=config)

In [ ]:
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)


In [26]:
# customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [27]:
# vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)